In [2]:
%autoreload 2

In [3]:
from collections import defaultdict
from argparse import Namespace
import copy
import gzip
import itertools
import os
import pickle
import sys
import typing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tatsu
import tatsu.ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.fitness_features import *
from src.ast_counter_sampler import *
from src.ast_mcmc_regrowth import *

In [4]:
fitness_df = utils.load_fitness_data()
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples.pddl']


,Index,src_file,game_name,domain_name,real,original_game_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,...,ast_ngram_n_5_0,ast_ngram_n_5_1,ast_ngram_n_5_2,ast_ngram_n_5_3,ast_ngram_n_5_4,ast_ngram_n_5_5,ast_ngram_n_5_6,ast_ngram_n_5_7,ast_ngram_n_5_8,ast_ngram_n_5_9
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,1,6172feb1665491d1efbce164-0,1,1,1,1.00,...,1,1,1,1,1,1,1,1,1,1
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,1,5f77754ba932fb2c4ba181d8-2,1,1,1,1.00,...,1,1,0,1,0,1,0,1,1,1
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,1,614b603d4da88384282967a7-3,1,1,1,0.00,...,1,0,0,0,0,0,0,0,0,0
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,1,5bc79f652885710001a0e82a-5,1,1,1,0.00,...,1,1,1,1,1,1,1,1,1,1
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,1,614dec67f6eb129c3a77defd-6,1,1,1,0.25,...,1,1,0,1,1,1,1,1,1,1


# Plan of attack
* Use the code in `ast_counter_sampler.py` to generate a sample from the MLE
* Score it with a fitness function adapted to working with a single example at a time
* At each subsequent iteration, regrow the game from a random node, and score the regrowth
* Since we have energy scores before and after, we can either accept greedily if $E_{new} < E_{old}$, or accept with probability $\exp(\beta (E_{old} - E_{new} ))$

In [5]:
DEFAULT_ARGS = argparse.Namespace(
    grammar_file=os.path.join('..', DEFAULT_GRAMMAR_FILE),
    parse_counter=False,
    counter_output_path=os.path.join('..', DEFAULT_COUNTER_OUTPUT_PATH),
    random_seed=DEFUALT_RANDOM_SEED,
)

In [6]:
FITNESS_MODEL_PATH = '../models/cv_fitness_model_2023_02_07.pkl.gz'
FITNESS_FEATURIZER_PATH = '../models/fitness_featurizer_2023_02_07.pkl.gz'

feature_names = [c for c in fitness_df.columns if c not in NON_FEATURE_COLUMNS]

mcmc = MCMCRegrowthSampler(DEFAULT_ARGS, feature_names,
    FITNESS_MODEL_PATH, FITNESS_FEATURIZER_PATH, greedy_acceptance=True, 
    plateau_patience_steps=100, max_steps=5000)  #   acceptance_temperature=10.0, 

In [7]:
# %prun -r 
n_samples = 10
mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=True)

for i in range(n_samples):
    print(i, mcmc.samples[i][2])

  0%|          | 0/10 [00:00<?, ?it/s]

Plateaued at step 391 with energy 5.492209434509277
Plateaued at step 664 with energy 4.709176540374756
Plateaued at step 262 with energy 1.2346500158309937
Plateaued at step 456 with energy 5.406106472015381
Plateaued at step 511 with energy 4.750307559967041
Plateaued at step 440 with energy 7.373414993286133
Plateaued at step 580 with energy 5.522914886474609
Plateaued at step 389 with energy 4.819112300872803
Plateaued at step 421 with energy 5.609058856964111
Plateaued at step 597 with energy 7.6075873374938965
0 5.492209434509277
1 4.709176540374756
2 1.2346500158309937
3 5.406106472015381
4 4.750307559967041
5 7.373414993286133
6 5.522914886474609
7 4.819112300872803
8 5.609058856964111
9 7.6075873374938965


In [8]:
mcmc.visualize_sample(2)

### Energy of visualized game: 1.235

### Top features contributing to the game's energy
feature name: **value** = (original feature value => regrown feature value) * weight

| Features increasing energy (= more fake)                             | Features decreasing energy (= more real)                                     |
|----------------------------------------------------------------------|------------------------------------------------------------------------------|
| text_ngram_n_5_score: **0.990** = (0.643) * 1.539                    | all_variables_used: **-1.848** = (1.000) * -1.848                            |
| text_ngram_n_4_score: **0.660** = (0.524) * 1.259                    | variable_not_repeated: **-1.619** = (1.000) * -1.619                         |
| text_ngram_n_2_9: **0.415** = (1.000) * 0.415                        | all_variables_defined: **-0.569** = (1.000) * -0.569                         |
| ast_ngram_n_2_1: **0.411** = (1.000) * 0.411                         | no_nested_logicals: **-0.479** = (1.000) * -0.479                            |
| ast_ngram_n_2_0: **0.394** = (1.000) * 0.394                         | all_preferences_used: **-0.415** = (1.000) * -0.415                          |
| text_ngram_n_3_score: **0.378** = (0.394) * 0.959                    | no_adjacent_same_modal: **-0.398** = (1.000) * -0.398                        |
| correct_predicate_function_arity: **0.375** = (1.000) * 0.375        | pref_forall_pref_forall_correct_types_correct: **-0.359** = (1.000) * -0.359 |
| ast_ngram_n_3_0: **0.373** = (1.000) * 0.373                         | no_two_number_operations: **-0.206** = (1.000) * -0.206                      |
| text_ngram_n_2_2: **0.361** = (1.000) * 0.361                        | pref_forall_pref_forall_correct_arity_correct: **-0.153** = (1.000) * -0.153 |
| ast_ngram_n_2_2: **0.350** = (1.000) * 0.350                         | ast_ngram_n_3_9: **-0.140** = (1.000) * -0.140                               |
| text_ngram_n_3_1: **0.340** = (1.000) * 0.340                        | max_depth_constraints_1: **-0.120** = (1.000) * -0.120                       |
| ast_ngram_n_2_4: **0.337** = (1.000) * 0.337                         | text_ngram_n_2_4: **-0.111** = (1.000) * -0.111                              |
| ast_ngram_n_2_5: **0.265** = (1.000) * 0.265                         | node_count_scoring_0: **-0.096** = (1.000) * -0.096                          |
| node_count_terminal_2: **0.212** = (1.000) * 0.212                   | on_arg_types_room_features_balls_setup: **-0.062** = (1.000) * -0.062        |
| no_identical_logical_children: **0.198** = (1.000) * 0.198           | setup_objects_used: **-0.043** = (0.500) * -0.086                            |
| text_ngram_n_2_score: **0.167** = (0.289) * 0.577                    | max_depth_terminal_2: **-0.032** = (1.000) * -0.032                          |
| agent_holds_arg_types_balls_constraints: **0.166** = (1.000) * 0.166 | no_adjacent_once: **-0.006** = (1.000) * -0.006                              |
| text_ngram_n_2_5: **0.141** = (1.000) * 0.141                        | ast_ngram_n_5_score: **-0.003** = (-0.006) * 0.506                           |
| mean_depth_constraints_1: **0.135** = (1.000) * 0.135                |                                                                              |
| mean_depth_setup_2: **0.125** = (1.000) * 0.125                      |                                                                              |

### Game:

```pddl
(define (game mcmc-2) (:domain medium-objects-room-v1)
(:setup
  (exists (?o - dodgeball)
    (game-conserved
      (on rug ?o)
    )
  )
)
(:constraints
  (and
    (forall (?ob ?t1 - dodgeball)
      (and
        (preference ballThrownFromDoor
          (then
            (once (agent_holds ?t1) )
            (hold (not (agent_holds ?ob) ) )
          )
        )
      )
    )
  )
)
(:terminal
  (>= (count ballThrownFromDoor:dodgeball) 40 )
)
(:scoring
  2
)
)
```

In [9]:
mcmc.visualize_sample(11)

### Energy of visualized game: 2.761

### Top features contributing to the game's energy
feature name: **value** = (original feature value => regrown feature value) * weight

| Features increasing energy (= more fake)                      | Features decreasing energy (= more real)                                          |
|---------------------------------------------------------------|-----------------------------------------------------------------------------------|
| text_ngram_n_5_score: **1.196** = (0.777) * 1.539             | all_variables_used: **-1.848** = (1.000) * -1.848                                 |
| text_ngram_n_4_score: **0.881** = (0.700) * 1.259             | variable_not_repeated: **-1.619** = (1.000) * -1.619                              |
| text_ngram_n_3_score: **0.480** = (0.501) * 0.959             | all_variables_defined: **-0.569** = (1.000) * -0.569                              |
| text_ngram_n_2_9: **0.415** = (1.000) * 0.415                 | no_nested_logicals: **-0.479** = (1.000) * -0.479                                 |
| ast_ngram_n_2_1: **0.411** = (1.000) * 0.411                  | all_preferences_used: **-0.415** = (1.000) * -0.415                               |
| ast_ngram_n_2_0: **0.394** = (1.000) * 0.394                  | no_two_number_operations: **-0.206** = (1.000) * -0.206                           |
| correct_predicate_function_arity: **0.375** = (1.000) * 0.375 | distance_arg_types_room_features_receptacles_setup: **-0.143** = (1.000) * -0.143 |
| ast_ngram_n_3_0: **0.373** = (1.000) * 0.373                  | node_count_scoring_0: **-0.096** = (1.000) * -0.096                               |
| text_ngram_n_2_2: **0.361** = (1.000) * 0.361                 | text_ngram_n_4_7: **-0.095** = (1.000) * -0.095                                   |
| ast_ngram_n_5_score: **0.355** = (0.701) * 0.506              | max_depth_terminal_2: **-0.032** = (1.000) * -0.032                               |
| text_ngram_n_3_1: **0.340** = (1.000) * 0.340                 | ast_ngram_n_5_2: **-0.002** = (1.000) * -0.002                                    |
| ast_ngram_n_4_score: **0.254** = (0.523) * 0.486              | ast_ngram_n_4_3: **-0.002** = (1.000) * -0.002                                    |
| ast_ngram_n_3_score: **0.245** = (0.603) * 0.406              |                                                                                   |
| text_ngram_n_2_score: **0.233** = (0.404) * 0.577             |                                                                                   |
| node_count_terminal_2: **0.212** = (1.000) * 0.212            |                                                                                   |
| no_identical_logical_children: **0.198** = (1.000) * 0.198    |                                                                                   |
| ast_ngram_n_3_6: **0.141** = (1.000) * 0.141                  |                                                                                   |
| ast_ngram_n_2_score: **0.134** = (0.529) * 0.253              |                                                                                   |
| mean_depth_setup_2: **0.125** = (1.000) * 0.125               |                                                                                   |
| max_depth_setup_2: **0.123** = (1.000) * 0.123                |                                                                                   |

### Game:

```pddl
(define (game mcmc-11) (:domain few-objects-room-v1)
(:setup
  (exists (?h - wall)
    (game-conserved
      (exists (?d - hexagonal_bin)
        (and
          (adjacent agent ?d)
          (< (distance ?h ?d) 1)
        )
      )
    )
  )
)
(:constraints
  (and
    (preference ballOnBedAtEnd
      (at-end
        (game_over )
      )
    )
  )
)
(:terminal
  (>= (count ballOnBedAtEnd) 3 )
)
(:scoring
  10
)
)
```

In [10]:
mcmc.visualize_sample(6)

### Energy of visualized game: 4.728

### Top features contributing to the game's energy
feature name: **value** = (original feature value => regrown feature value) * weight

| Features increasing energy (= more fake)                      | Features decreasing energy (= more real)                                      |
|---------------------------------------------------------------|-------------------------------------------------------------------------------|
| text_ngram_n_5_score: **1.139** = (0.740) * 1.539             | all_variables_used: **-1.848** = (1.000) * -1.848                             |
| text_ngram_n_4_score: **0.848** = (0.674) * 1.259             | variable_not_repeated: **-1.619** = (1.000) * -1.619                          |
| text_ngram_n_3_score: **0.518** = (0.540) * 0.959             | all_variables_defined: **-0.569** = (1.000) * -0.569                          |
| text_ngram_n_2_9: **0.415** = (1.000) * 0.415                 | no_nested_logicals: **-0.479** = (1.000) * -0.479                             |
| ast_ngram_n_2_1: **0.411** = (1.000) * 0.411                  | in_arg_types_receptacles_balls_constraints: **-0.474** = (1.000) * -0.474     |
| ast_ngram_n_2_0: **0.394** = (1.000) * 0.394                  | no_adjacent_same_modal: **-0.398** = (1.000) * -0.398                         |
| ast_ngram_n_3_8: **0.386** = (1.000) * 0.386                  | pref_forall_pref_forall_correct_types_correct: **-0.359** = (1.000) * -0.359  |
| ast_ngram_n_5_0: **0.385** = (1.000) * 0.385                  | no_two_number_operations: **-0.206** = (1.000) * -0.206                       |
| ast_ngram_n_4_0: **0.382** = (1.000) * 0.382                  | pref_forall_pref_forall_correct_arity_correct: **-0.153** = (1.000) * -0.153  |
| ast_ngram_n_2_7: **0.376** = (1.000) * 0.376                  | max_depth_constraints_1: **-0.120** = (1.000) * -0.120                        |
| correct_predicate_function_arity: **0.375** = (1.000) * 0.375 | node_count_scoring_0: **-0.096** = (1.000) * -0.096                           |
| ast_ngram_n_3_0: **0.373** = (1.000) * 0.373                  | text_ngram_n_4_7: **-0.095** = (1.000) * -0.095                               |
| ast_ngram_n_4_1: **0.367** = (1.000) * 0.367                  | text_ngram_n_5_2: **-0.068** = (1.000) * -0.068                               |
| text_ngram_n_2_2: **0.361** = (1.000) * 0.361                 | adjacent_arg_types_receptacles_furniture_setup: **-0.063** = (1.000) * -0.063 |
| ast_ngram_n_3_1: **0.361** = (1.000) * 0.361                  | max_depth_terminal_2: **-0.032** = (1.000) * -0.032                           |
| text_ngram_n_3_0: **0.359** = (1.000) * 0.359                 | setup_objects_used: **-0.029** = (0.333) * -0.086                             |
| ast_ngram_n_2_2: **0.350** = (1.000) * 0.350                  | no_adjacent_once: **-0.006** = (1.000) * -0.006                               |
| text_ngram_n_3_1: **0.340** = (1.000) * 0.340                 |                                                                               |
| text_ngram_n_2_3: **0.329** = (1.000) * 0.329                 |                                                                               |
| text_ngram_n_4_0: **0.244** = (1.000) * 0.244                 |                                                                               |

### Game:

```pddl
(define (game mcmc-6) (:domain medium-objects-room-v1)
(:setup
  (forall (?b - ball)
    (exists (?b - hexagonal_bin)
      (game-conserved
        (adjacent ?b bed)
      )
    )
  )
)
(:constraints
  (and
    (preference basketMadeFromRug
      (exists (?c1 - block)
        (at-end
          (on ?c1 bed)
        )
      )
    )
    (forall (?b2 - dodgeball)
      (and
        (preference ballThrownToBin
          (exists (?l4 - doggie_bed)
            (then
              (hold (in ?l4 ?b2) )
              (any)
              (hold (not (toggled_on ?l4) ) )
            )
          )
        )
      )
    )
  )
)
(:terminal
  (>= (count-once-per-objects ballThrownToBin:dodgeball) 3 )
)
(:scoring
  (total-score)
)
)
```

### Notes from looking at these games
* **TODO:** debug the 'section-without' logic, it seems to not work
* **TODO:** add features for preferences with different lengths of modals

In [2]:
print(ast_printer.ast_to_string(samples[8][0], '\n'))

NameError: name 'ast_printer' is not defined

In [ ]:
fitness_featurizer = build_fitness_featurizer(DEFAULT_ARGS)
features = fitness_featurizer.parse(samples[7][0], 'mcmc', True)
{k: v for k, v in features.items() if 'correctly' in k or 'forall' in k or 'no' in k}

In [ ]:
with gzip.open(DEFAULT_FITNESS_FUNCTION_PATH, 'rb') as f:
    cv_fitness_model = pickle.load(f)

In [ ]:
weights = cv_fitness_model.named_steps['fitness'].model.fc1.weight.data.detach().numpy().squeeze()
bias = cv_fitness_model.named_steps['fitness'].model.fc1.bias.data.detach().numpy().squeeze()
print(weights.mean(), bias)

In [ ]:
plt.hist(weights, bins=100)
plt.title('Fitness Model Weights')
plt.xlabel('Weight magnitude')
plt.ylabel('Count')

In [12]:
import ast_crossover_sampler
asts = [ast for ast in cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', 
    mcmc.grammar_parser, False, relative_path='..')]

mcmc_with_crossover = MCMCRegrowthCrossoverSampler(DEFAULT_ARGS, 
    ast_crossover_sampler.CrossoverType.SAME_RULE, asts, 0.25,
    '../models/cv_fitness_model_2022_12_26.pkl.gz', 
    greedy_acceptance=True, plateau_patience_steps=20, acceptance_temperature=10.0, max_steps=1000)

In [13]:
mcmc_with_crossover.multiple_samples(10, verbose=2, should_tqdm=True)
print([x[2] for x in mcmc_with_crossover.samples])

  0%|          | 0/10 [00:00<?, ?it/s]

Sampling exception, skipping sample
Regrowth generated identical games, repeating
Accepted step 1 with fitness 7.857532501220703
Accepted step 2 with fitness 7.784302711486816
Accepted step 3 with fitness 7.775148391723633
Accepted step 4 with fitness 7.692765235900879
Accepted step 6 with fitness 7.536055564880371
Regrowth generated identical games, repeating
Accepted step 8 with fitness 7.5235395431518555
Accepted step 10 with fitness 7.4317522048950195
Accepted step 11 with fitness 6.490839004516602
Accepted step 17 with fitness 5.810407638549805
Accepted step 19 with fitness 5.400604248046875
Accepted step 28 with fitness 5.354710578918457
Accepted step 44 with fitness 4.757689476013184
Accepted step 49 with fitness 4.688848495483398
Accepted step 68 with fitness 4.574114799499512
Accepted step 75 with fitness 4.34464693069458
Regrowth generated identical games, repeating
Accepted step 93 with fitness 3.9308552742004395
Regrowth generated identical games, repeating
Accepted step 10

In [14]:
print([x[2] for x in mcmc_with_crossover.samples])

[-0.5954062938690186, -0.4696420133113861, 1.7651313543319702, 5.2624006271362305, 0.8746525049209595, 1.2288413047790527, 0.9395236968994141, 1.8448952436447144, 0.8919482231140137, 1.2082358598709106]
